In [ ]:
import math

def gives_angle(perp_distance, eyes_distance):
    hypotenuse = math.sqrt(perp_distance**2 + eyes_distance**2)
    angle = math.atan(perp_distance/eyes_distance)
    return angle

    


In [14]:
import cv2
import  math
# Load the Haar cascade for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


# Initialize the video capture
cap = cv2.VideoCapture(0)
eye_coordinates = []


REFERENCE_OBJECT_WIDTH = 20  # Width of the reference object in centimeters
FOCAL_LENGTH = 550  # Focal length of the camera in pixels

def calculate_distance_to_object(object_width, focal_length, measured_width):
    # Calculate the distance to the object using the formula: distance = (reference_width * focal_length) / measured_width
    distance = (object_width * focal_length) / measured_width
    return distance

screen_width = 1920
screen_height = 1080

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,1.3,10)
    for (x,y,w,h) in faces:
        distance = calculate_distance_to_object(REFERENCE_OBJECT_WIDTH, FOCAL_LENGTH, w)
        cv2.putText(frame, f"Distance: {distance:.2f} cm", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to the grayscale frame
    blurred_gray = cv2.GaussianBlur(gray, (5, 5), 0)
    eyes = eye_cascade.detectMultiScale(blurred_gray, scaleFactor=1.3, minNeighbors=10)
    eye_coordinates = []
    
        

    if len(eyes) >= 2:

        # Iterate over the detected eyes
        for (ex, ey, ew, eh) in eyes[:2]:
            # Extract the eye region from the frame
            eye = frame[ey:ey+eh, ex:ex+ew]

            # Convert the eye region to grayscale
            eye_gray = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(eye_gray, 30, 255, cv2.THRESH_BINARY)

            # Find contours in the binary image
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Iterate over the contours
            for contour in contours:
                # Calculate the area of the contour
                area = cv2.contourArea(contour)

                # Filter contours based on area to find the dark portion of the eyeball
                if area > 100:
                    # Find the centroid of the contour using the moments
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])

                        # Draw a red dot at the center of the dark portion of the eyeball
                        cv2.circle(eye, (cx, cy), 2, (0, 0, 255), -1)


            # Append the coordinates of the red dot to the list
            eye_coordinates.append([ex+cx, ey+cy])
            
            # Replace the eye region back in the frame
            frame[ey:ey+eh, ex:ex+ew] = eye
        


        eye_coordinates = sorted(eye_coordinates, key=lambda x: x[0])

        left_eye_center_x = eye_coordinates[0][0]
        right_eye_center_x = eye_coordinates[1][0]
        left_eye_center_y = eye_coordinates[0][1]
        right_eye_center_y = eye_coordinates[1][1]

        
        
        # Calculate the average horizontal angle
        horizontal_angle = (math.atan((left_eye_center_x - screen_width / 2) / screen_width) +
                            math.atan((right_eye_center_x - screen_width / 2) / screen_width)) / 2 * 2

        # Calculate the average vertical angle
        vertical_angle = (math.atan((left_eye_center_y - screen_height / 2) / screen_height) +
                        math.atan((right_eye_center_y - screen_height / 2) / screen_height)) / 2 * 2

        # Calculate the horizontal distance from the center of the screen
        horizontal_distance = distance * math.tan(horizontal_angle)

        # Calculate the vertical distance from the center of the screen
        vertical_distance = distance * math.tan(vertical_angle)

        # Calculate the X coordinate on the screen
        x_coordinate = int((horizontal_distance / distance) * screen_width / 2 + screen_width / 2)

        # Calculate the Y coordinate on the screen
        y_coordinate = int((vertical_distance / distance) * screen_height / 2 + screen_height / 2)


        angle = 180-math.atan((distance)/())* (180 / math.pi)
        
        print(angle)


        
        
            
        # print(eye_coordinates)
        
        
        # print(eye_coordinates)
    cv2.imshow('Video', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()


85.77818701335069
85.77818701335069
85.23463405189528
85.34327903544337
85.88699217428626
85.88699217428626
85.88699217428626
86.1772824862698
86.52225648744218
86.37697205482255
86.7947840689865
86.7947840689865
86.94019228538524
86.94019228538524
87.24931345862744
86.92201406212217
86.92201406212217
86.83113232189284
86.7402665407712
86.7402665407712
86.7402665407712
86.7402665407712
86.7402665407712
86.7402665407712
86.7402665407712
86.7402665407712
85.50630921847333
85.50630921847333
85.50630921847333
85.50630921847333
85.50630921847333
85.39761416456591
85.39761416456591
85.39761416456591
85.39761416456591
85.39761416456591
85.23463405189528
85.48819110394243
85.48819110394243
85.63316103744349
85.63316103744349
85.81445204995839
85.81445204995839
85.9958270592906
86.08654494465341
86.08654494465341
86.32250231719999
86.32250231719999
86.23173425952419
86.23173425952419
86.68575492083315
86.88565947993348
86.88565947993348


In [3]:
import cv2

# Constants for reference object size and distance
REFERENCE_OBJECT_WIDTH = 20  # Width of the reference object in centimeters
FOCAL_LENGTH = 550  # Focal length of the camera in pixels

def calculate_distance_to_object(object_width, focal_length, measured_width):
    # Calculate the distance to the object using the formula: distance = (reference_width * focal_length) / measured_width
    distance = (object_width * focal_length) / measured_width
    return distance

# Start the webcam
cap = cv2.VideoCapture(0)

# Load the face cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    # Read the current frame from the webcam
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, 1.3, 10)

    # Iterate over detected faces
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        
        # Calculate the distance to the person's face
        distance = calculate_distance_to_object(REFERENCE_OBJECT_WIDTH, FOCAL_LENGTH, w)

        # Display the distance on the frame
        cv2.putText(frame, f"Distance: {distance:.2f} cm", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Distance Estimation', frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


In [9]:
import cv2

# Load the Haar cascade for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize the video capture
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    if ret:
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to the grayscale frame
        blurred_gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # Detect eyes in the blurred grayscale frame
        eyes = eye_cascade.detectMultiScale(blurred_gray, scaleFactor=1.3, minNeighbors=10)

        # Check if both eyes are detected
        if len(eyes) == 2:
            # Extract the eye centers and sort them based on x-coordinate and y-coordinate
            eye_centers = sorted([(ex + ew // 2, ey + eh // 2) for (ex, ey, ew, eh) in eyes])

            left_eye_center = eye_centers[0]
            right_eye_center = eye_centers[1]

            # Compare the relative positions of eye centers to determine the gaze direction
            horizontal_gaze = "Right" if right_eye_center[0] > left_eye_center[0] else "Left"
            vertical_gaze = "Up" if right_eye_center[1] < left_eye_center[1] else "Down"

            # Draw circles around the eye centers on the frame
            cv2.circle(frame, left_eye_center, 2, (0, 0, 255), -1)
            cv2.circle(frame, right_eye_center, 2, (0, 0, 255), -1)

            # Display the gaze direction on the frame
            cv2.putText(frame, f"Horizontal Gaze: {horizontal_gaze}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Vertical Gaze: {vertical_gaze}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Video', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()
